In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../Dataset/metflix.csv")
df.shape

(540, 7)

In [4]:
df.dtypes

ids           int64
ages          int64
bookmarks     int64
replies       int64
ratings       int64
watches       int64
period       object
dtype: object

In [5]:
df.head()

,ids,ages,bookmarks,replies,ratings,watches,period
0,912,47,5,11,23,74,16M
1,325,44,6,4,11,90,10M
2,282,49,6,12,38,188,10M
3,1234,21,13,53,89,766,21M
4,1279,25,40,73,186,736,22M


In [6]:
df['period'] = df['period'].str.replace("M", "")
df.head()

,ids,ages,bookmarks,replies,ratings,watches,period
0,912,47,5,11,23,74,16
1,325,44,6,4,11,90,10
2,282,49,6,12,38,188,10
3,1234,21,13,53,89,766,21
4,1279,25,40,73,186,736,22


In [7]:
df['period'] = df['period'].astype(int)

In [8]:
df.dtypes

ids          int64
ages         int64
bookmarks    int64
replies      int64
ratings      int64
watches      int64
period       int32
dtype: object

In [9]:
customer = df.copy()

In [11]:
median_ = customer['ratings'].median()
median_

99.5

In [12]:
q1 = np.quantile(customer['ratings'], 0.25)
q3 = np.quantile(customer['ratings'], 0.75)
print(q1, q3)

24.75 215.25


In [13]:
IQR=q3-q1
print(IQR)

190.5


In [14]:
lower=median_-2.3*IQR
upper = median + 2.3*IQR
print(lower, upper)

-338.65 537.65


In [18]:
sub1 = customer.loc[(lower<customer['ratings']) & (customer['ratings']<upper)]

In [21]:
answer = sub1['ratings'].mean()
answer = round(answer, 2)
answer

126.77

In [24]:
sub2 = customer.copy()
sub2['age_cls'] = sub2['ages']//10
sub2.head()

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
0,912,47,5,11,23,74,16,4
1,325,44,6,4,11,90,10,4
2,282,49,6,12,38,188,10,4
3,1234,21,13,53,89,766,21,2
4,1279,25,40,73,186,736,22,2


In [39]:
means = sub2.groupby('age_cls')['replies'].mean()
A=means.max()

In [40]:
B=means.min()

In [41]:
answer = A+B
answer = round(answer, 2)
answer

82.29

In [42]:
customer.head()

,ids,ages,bookmarks,replies,ratings,watches,period
0,912,47,5,11,23,74,16
1,325,44,6,4,11,90,10
2,282,49,6,12,38,188,10
3,1234,21,13,53,89,766,21
4,1279,25,40,73,186,736,22


In [43]:
sub3=customer.copy()
sub3['age_cls'] = np.where(sub3['ages']>=40, 1, 0)

In [44]:
sub3.head()

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
0,912,47,5,11,23,74,16,1
1,325,44,6,4,11,90,10,1
2,282,49,6,12,38,188,10,1
3,1234,21,13,53,89,766,21,0
4,1279,25,40,73,186,736,22,0


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
random_state = 229

In [47]:
train_set, test_set = train_test_split(sub3, random_state=random_state, test_size = 0.01)
train_set.shape

(534, 8)

In [48]:
test_set.shape

(6, 8)

In [49]:
test_set

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
503,2145,37,34,41,50,123,36,0
397,95,26,19,94,161,782,10,0
524,1241,35,20,22,70,831,21,0
32,635,24,49,68,187,430,12,0
341,855,21,26,91,303,456,15,0
225,513,52,0,8,29,74,10,1


In [51]:
train_set.loc[train_set['replies']==0]

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
278,1334,24,15,0,0,647,23,0
208,249,48,1,0,0,170,10,1
173,1359,40,0,0,0,102,23,1
279,1711,43,3,0,0,76,28,1
477,400,43,5,0,0,34,25,1
168,875,41,2,0,0,188,15,1
440,509,41,2,0,0,21,10,1
487,75,50,5,0,0,6,10,1
82,1326,45,4,0,0,9,34,1
147,503,44,1,0,0,167,10,1


In [54]:
len(train_set.loc[train_set['replies']==0])

13

In [55]:
#단계3

In [60]:
sub3_1 = train_set.copy()
sub3_1['X'] = sub3_1['period']/(sub3_1['replies']+0.01)
sub3_1.head()

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls,X
336,195,48,2,7,20,190,10,1,1.426534
278,1334,24,15,0,0,647,23,0,2300.000000
149,500,49,0,4,6,84,10,1,2.493766
241,2130,23,46,102,103,716,36,0,0.352907
413,2140,42,2,1,3,80,36,1,35.643564


In [61]:
model1 = KNeighborsClassifier(weights='distance', p=1)
model1.fit(sub3_1[['X']], sub3_1['age_cls'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                     weights='distance')

In [62]:
sub3_2 = train_set.loc[sub3['replies']!=0]
len(sub3_2)

521

In [65]:
sub3_2['X'] = sub3_2['period']/sub3_2['replies']
sub3_2.head()

C:\Users\syoun\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ids,ages,bookmarks,replies,ratings,watches,period,age_cls,X
336,195,48,2,7,20,190,10,1,1.428571
149,500,49,0,4,6,84,10,1,2.500000
241,2130,23,46,102,103,716,36,0,0.352941
413,2140,42,2,1,3,80,36,1,36.000000
109,341,18,17,52,187,331,10,0,0.192308


In [66]:
model2 = KNeighborsClassifier(weights='distance', p=1)
model2.fit(sub3_2[['X']], sub3_2['age_cls'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                     weights='distance')

In [67]:
test_set.head()

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
503,2145,37,34,41,50,123,36,0
397,95,26,19,94,161,782,10,0
524,1241,35,20,22,70,831,21,0
32,635,24,49,68,187,430,12,0
341,855,21,26,91,303,456,15,0


In [68]:
test_set['X1'] = test_set['period']/(test_set['replies']+0.01)
test_set['X2'] = test_set['period']/test_set['replies']
test_set.head()

C:\Users\syoun\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\syoun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ids,ages,bookmarks,replies,ratings,watches,period,age_cls,X1,X2
503,2145,37,34,41,50,123,36,0,0.877835,0.878049
397,95,26,19,94,161,782,10,0,0.106372,0.106383
524,1241,35,20,22,70,831,21,0,0.954112,0.954545
32,635,24,49,68,187,430,12,0,0.176445,0.176471
341,855,21,26,91,303,456,15,0,0.164817,0.164835


In [71]:
pred1 = model1.predict(test_set[['X1']])
pred2 = model2.predict(test_set[['X2']])
print(pred1)
print(pred2)
print(test_set['age_cls'])

[0 0 0 0 0 1]
[0 0 0 0 0 1]
503    0
397    0
524    0
32     0
341    0
225    1
Name: age_cls, dtype: int32


In [72]:
answer = 1